# Notebook 2: Conducting and Evaluating Regression Analysis

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import glob
import pandas as pd

Import datasets that were preprocessed in Notebook 1

In [2]:
wb_data = pd.read_csv("data/wb_data.csv")
wb_data_short = pd.read_csv("data/wb_data_short.csv")
whr_data = pd.read_csv("data/whr_data.csv")